- CRSS where each sample corresponds to one person, not to one accident.  
- This one corresponds better to evaluating per phone.

In [1]:
%%latex
\tableofcontents

<IPython.core.display.Latex object>

# Setup
## Import Libraries

In [2]:
import sys, copy, math, time

print ('Python version: {}'.format(sys.version))

from IPython.display import display, HTML

from collections import Counter

import numpy as np
print ('NumPy version: {}'.format(np.__version__))
np.set_printoptions(suppress=True)

import pandas as pd
print ('Pandas version:  {}'.format(pd.__version__))
pd.set_option('display.max_rows', 500)



Python version: 3.9.7 (default, Oct 22 2021, 13:24:00) 
[Clang 13.0.0 (clang-1300.0.29.3)]
NumPy version: 1.21.3
Pandas version:  1.2.4


## Import Data

### Vehicle.csv from CRSS

In [3]:
def Import_Data_Vehicle():
    print ('Import_Data_Vehicle()')

    df = pd.DataFrame([])
    for year in ['2016','2017','2018']:
        filename = '../../Big_Files/CRSS_2020_Update/CRSS' + year + 'CSV/VEHICLE.CSV'
#        filename = '../../CRSS/CRSS' + year + 'CSV/VEHICLE.CSV'
        temp = pd.read_csv(filename, index_col=None, low_memory=False)
        print (year, len(temp))
        df = df.append(temp)

    for year in ['2019','2020']:
        filename = '../../Big_Files/CRSS_2020_Update/CRSS' + year + 'CSV/vehicle.csv'
#        filename = '../../CRSS/CRSS' + year + 'CSV/vehicle.csv'
        temp = pd.read_csv(filename, index_col=None, encoding='latin1', low_memory=False)
        print (year, len(temp))
        df = df.append(temp)

    print (df.shape)
    print ()
    return df

### person.csv from CRSS

In [4]:
def Import_Data_Person():
    print ('Import_Data_Person()')

    df = pd.DataFrame([])
    for year in ['2016','2017','2018']:
        filename = '../../Big_Files/CRSS_2020_Update/CRSS' + year + 'CSV/PERSON.CSV'
#        filename = '../../CRSS/CRSS' + year + 'CSV/PERSON.CSV'
        temp = pd.read_csv(filename, index_col=None)
        print (year, len(temp))
        df = df.append(temp)

    for year in ['2019','2020']:
        filename = '../../Big_Files/CRSS_2020_Update/CRSS' + year + 'CSV/person.csv'
#        filename = '../../CRSS/CRSS' + year + 'CSV/person.csv'
        temp = pd.read_csv(filename, index_col=None, encoding='latin1')
        print (year, len(temp))
        df = df.append(temp)

#    for feature in df:
#        print (feature)
    

#    for feature in df:
#        if 'NAME' in feature:
#            df.drop(columns=[feature], inplace=True)

    print (df.shape)
    print ()
    return df

### All Data

In [5]:
def Import_Data():
    print ('Import_Data()')
    df_Vehicle = Import_Data_Vehicle()
    df_Person = Import_Data_Person()
    
    print ()
    return df_Vehicle, df_Person

# Build Database

In [6]:
def Build_Vehicle_Dataset(df_Vehicle):
    print ('Build_Vehicle_Dataset()')
    data = pd.DataFrame()

    F = [
        'ACC_TYPE',
        'BDYTYP_IM',
        'BODY_TYP',
        'BUS_USE',
        'CARGO_BT',
        'DEFORMED',
        'DR_PRES',
        'EMER_USE',
        'FIRE_EXP',
        'HAZ_CNO',
        'HAZ_INV',
        'HAZ_PLAC',
        'HAZ_REL',
        'HIT_RUN',
#        'HITRUN_IM',
        'IMPACT1',
        'IMPACT1_IM',
        'J_KNIFE',
        'M_HARM',
        'MAK_MOD',
        'MAKE',
        'MAX_VSEV',
        'MDLYR_IM',
        'MOD_YEAR',
        'MODEL',
        'MXVSEV_IM',
        'NUM_INJV',
        'NUMINJ_IM',
        'NUMOCCS',
        'P_CRASH1',
        'P_CRASH2',
        'P_CRASH3',
        'PCRASH1_IM',
        'PCRASH4',
        'PCRASH5',
        'ROLINLOC',
        'ROLLOVER',
        'SPEC_USE',
        'SPEEDREL',
        'TOW_VEH',
        'TOWED',
        'TRAV_SP',
        'V_ALCH_IM',
        'VALIGN',
        'VEH_ALCH',
        'VEVENT_IM',
        'VNUM_LAN',
        'VPROFILE',
        'VSPD_LIM',
        'VSURCOND',
        'VTCONT_F',
        'VTRAFCON',
        'VTRAFWAY',
        'CASENUM',
        'VEH_NO',
    ]

    for f in F:
        data[f] = df_Vehicle[f]
    
    
    print ()
    return data

## Build_Person_Dataset

In [7]:
def Build_Person_Dataset(df_Person):
    print ('Build_Person_Dataset')
    data = pd.DataFrame()
    
    F = [
        'CASENUM',
        'VEH_NO',
#        'AGE_IM',
#        'SEX_IM',
#        'MAKE',
#        'BODY_TYP',
#        'PER_TYP',
#        'MOD_YEAR',
#        'MAK_MOD',
    ]

    for f in F:
        data[f] = df_Person[f]
        
    data['HOSPITAL'] = df_Person['HOSPITAL'].apply(lambda x:1 if x in [1,2,3,4,5] else 0)

    data = data[data.VEH_NO != 0]

    data = data.reindex(sorted(data.columns), axis=1)    
    
    print ()
    return data

## Merge Datasets

In [8]:
def Build_Dataset(df_Vehicle, df_Person):
    print ('Build_Dataset()')
    print ()
    data_Vehicle = Build_Vehicle_Dataset(df_Vehicle)
    data_Person = Build_Person_Dataset(df_Person)
    
    
    
    U = data_Vehicle.CASENUM.unique()
    V = data_Person.CASENUM.unique()
    U = set(U)
    V = set(V)
    print ('Do all of the cases in data_Person appear in data_Vehicle?')
    print (V.issubset(U))
    print ()
    
    
    
#    data = pd.merge(
#        data_Vehicle, data_Person, 
#        on=['CASENUM', 'VEH_NO'],
#        how="outer", sort=False
#    )
    
    data = pd.merge(
        data_Person, data_Vehicle, 
        left_on=['CASENUM', 'VEH_NO'], right_on = ['CASENUM', 'VEH_NO'], 
        how="inner", sort=False
    )
    
    print ('data_Vehicle.shape')
    print (data_Vehicle.shape)
    print ('data_Person.shape')
    print (data_Person.shape)
    print ('data.shape')
    print (data.shape)
    

#    data.drop(columns=['CASENUM'], inplace=True)
#    data.astype(bool)

    print (data.head())

    return data

## Feature Names

In [9]:
def Feature_Names(data, Named_Features):
    print ('Feature_Names')
    D = {}
    for f in Named_Features:
        g = f + 'NAME'
        A = pd.concat([data[f],data[g]], axis=1)
        A.drop_duplicates(inplace=True)
        A.dropna(inplace=True)
#        print (f)
#        print (len(A))
#        print (A.head())
#        print ()
        B = dict(zip(A[f],A[g]))
        D[f] = B
#        print (B)
#        print ()
#    print (D)
    print ()
    return D
        

In [10]:
def Remove_Unknowns_in_Feature(data, feature):
    
    Unknowns = {
        'HOSPITAL': [],
        'ACC_TYPE': [99],
        'BDYTYP_IM': [],
        'BODY_TYP': [98, 99, 49, 79],
        'BUS_USE': [98, 99],
        'CARGO_BT': [98, 99],
        'DEFORMED': [8, 9],
        'DR_PRES': [9],
        'EMER_USE': [8, 9],
        'FIRE_EXP': [],
        'HAZ_CNO': [88],
        'HAZ_INV': [],
        'HAZ_PLAC': [8],
        'HAZ_REL': [8],
        'HIT_RUN': [9],
        'IMPACT1': [98, 99],
        'IMPACT1_IM': [],
        'J_KNIFE': [],
        'M_HARM': [98, 99],
        'MAK_MOD': [],
        'MAKE': [99],
        'MAX_VSEV': [9],
        'MDLYR_IM': [],
        'MOD_YEAR': [9998, 9999],
        'MODEL': [],
        'MXVSEV_IM': [],
        'NUM_INJV': [99],
        'NUMINJ_IM': [],
        'NUMOCCS': [99],
        'P_CRASH1': [99],
        'P_CRASH2': [99],
        'P_CRASH3': [99],
        'PCRASH1_IM': [],
        'PCRASH4': [9],
        'PCRASH5': [9],
        'ROLINLOC': [9],
        'ROLLOVER': [],
        'SPEC_USE': [98, 99],
        'SPEEDREL': [9],
        'TOW_VEH': [9],
        'TOWED': [8, 9],
        'TRAV_SP': [998, 999],
        'V_ALCH_IM': [],
        'VALIGN': [8, 9],
        'VEH_ALCH': [9],
        'VEVENT_IM': [],
        'VNUM_LAN': [8, 9],
        'VPROFILE': [8, 9],
        'VSPD_LIM': [98, 99],
        'VSURCOND': [98, 99],
        'VTCONT_F': [8, 9],
        'VTRAFCON': [97, 99],
        'VTRAFWAY': [8, 9],
        'CASENUM': [],
        'VEH_NO': [],
    }
    
#    print ('Remove_Unknowns_in_Feature ', feature, Unknowns[feature], len(data))
    data_temp = data[~data[feature].isin(Unknowns[feature])]
#    print ('Remove_Unknowns_in_Feature ', feature, Unknowns[feature], len(data_temp))
#    print ()
    
    return data_temp, Unknowns[feature]

## Correlation

In [11]:
def Correlation(data, target, feature, value, name):
    TN = 0
    FP = 0
    FN = 0
    TP = 0
    contingency_matrix = pd.crosstab(data[target], data[feature])
    cm = contingency_matrix.values.tolist()
    if len(cm)==2 and len(cm[0])==2:
        corr = cm[1][1] / (cm[0][1] + cm[1][1])
        per = (cm[0][1] + cm[1][1])/(cm[0][0] + cm[0][1] + cm[1][0] + cm[1][1])
    else:
        corr = 0
        per = 0
    per = round(per*100,4)
    corr = round(corr*100,4)
#    print ("    - ", feature)
#    print ("    - ", value)
#    print ("    - ", name)
#    print (contingency_matrix)
#    print ('        - per = ', per)
#    print ("        - corr = ", corr)
#    print ()
    return (per, corr)

def Correlation_by_Value(data, target, feature, Feature_Names_Dict, Unknowns):
# I decided against the np.unique because it treats each nan as a separate entry.
#    V = np.unique(data[feature].values) 
    V = data[feature].unique()
#    print (V)
    B = []

    for value in V:
        A = pd.DataFrame()
        A[feature] = data[feature].apply(lambda x: 1 if x==value else 0)
        A[target] = data[target]
        if feature in Feature_Names_Dict:
            if value in Feature_Names_Dict[feature]:
                name = Feature_Names_Dict[feature][value]
            else:
                name=str(value)
        else:
            name = str(value)
#        if len(name)>30:
#            name = name[:30]
        per, corr = Correlation(A, target, feature, value, name)
        B.append([feature, value, name, per, corr])
#    print (feature)
    B = sorted(B, key=lambda x:x[4], reverse=True)
    for b in B:
        c = b[1]
        try:
            c = int(c)
        except:
            c=c
        else:
            c = int(c)
#        print (c, end=',')
#    print ()
#    print ()

    # Print grouped into 100/p blocks of same size
    print ("    feature = '%s'" % feature)
    print ('    A = [')
    p = 20
    s = 0.0
    s2 = 0.0
    n=0
    print ("        ['%d', [" % n , end='')
    for b in B:
        t = s + b[3]
        if b[3]<10:
            s2 = s2 + b[3]
        q = int(s/p)
        r = int((t-0.001)/p)
        if r>q or b[3]>10:
            print ("]], # ", round(s2,4), '%')
            s2 = 0.0
            n += 1
            print ("        ['%d', [" % n , end='')
        s = t
        
        c = b[1]
        try:
            c = int(c)
        except:
            c=c
        else:
            c = int(c)
        print (c, end=',')
        if b[3]>10:
            print ("]], # ", round(b[3],4), '%')
            s2=0.0
            n += 1
            print ("        ['%d', [" % n , end='')
    print ("]], # ", round(s2,4), '%')
    print ("        ['Unknowns', [", end='')
    for u in Unknowns:
        print (u, end=', ')
    print ("]]" )
    print ('    ]')
    print ('    data = Build_Individual_Feature_with_Dict(df_Veh, data, feature, A)')
    print ()
    
    C = pd.DataFrame(B)
    C.columns = ['Feature', 'Code', 'Name', 'Per', 'Corr']
#    C.drop(C[C['Per'] < 0.1].index, inplace=True)
#    print (C)
    display(C)

    TeX = open('../Correlation/Correlation_' + feature + '.tex', 'w')
    E = [c for c in B if c[3]>=0.0]
    
        
    
    for c in E:
        a = c[0]
        b = c[1]
        d = c[2]
        e = "{:.4f}".format(c[3])
        f = "{:.4f}".format(c[4])
        TeX.write('\t & \\verb|%s| & %s & %s & %s & %s \\cr\n' % (a,b,d,e,f))
    

    TeX = open('../Correlation/Correlation_Ordered_' + feature + '.tex', 'w')
    E = sorted(B, key=lambda x:x[1], reverse=False)

    
    for c in E:
        a = c[0]
        b = c[1]
        d = c[2]
        e = "{:.4f}".format(c[3])
        f = "{:.4f}".format(c[4])
        TeX.write('\t & \\verb|%s| & %s & %s & %s & %s \\cr\n' % (a,b,d,e,f))
    

    print ()
    return B

def Correlation_All(data, target, Feature_Names_Dict):
    print ('Correlation_All')
    
    C = []
    for feature in data:
        data_temp, Unknowns = Remove_Unknowns_in_Feature(data, feature)
        U = data_temp[feature].unique()
#        print (feature, len(U))
        if len(U)<10000:
            B = Correlation_by_Value(
                data_temp, target, feature, Feature_Names_Dict, Unknowns
            )
            for b in B:
                C.append(b)
#            print ()
#        print ()
#    for c in C:
#        print (c)
#    print ()
    C = sorted(C, key=lambda x:x[4], reverse=True)
    D = pd.DataFrame(C)
    D.columns = ['Feature', 'Code', 'Name', 'Per', 'Corr']
    print (D)
    print ()
    
    D.drop(D[D['Per'] < 0.5].index, inplace=True)
    print (D)
    print ()
    
    TeX = open('../Correlation/Correlation.tex', 'w')
    E = [c for c in C if c[3]>=0.5]
    
    for c in E:
        a = c[0]
        b = c[1]
        d = c[2]
        e = "{:.4f}".format(c[3])
        f = "{:.4f}".format(c[4])
        TeX.write('\\verb|%s| & %s & %s & %s & %s \\cr\n' % (a,b,d,e,f))
    
    return 0

    

# Main()

In [12]:
def Main():
    target = 'HOSPITAL'
    df_Vehicle, df_Person = Import_Data()
    
    """
    print ('Features in df_Vehicle, with Number of Unique Values and Number of Blank Values')
    for feature in df_Vehicle:
        U = df_Vehicle[feature].unique()
        s = df_Vehicle[feature].isna().sum()
        print (feature, len(U), s)
    print ()

    print ('Features in df_Person, with Number of Unique Values and Number of Blank Values')
    for feature in df_Person:
        U = df_Person[feature].unique()
        s = df_Person[feature].isna().sum()
        print (feature, len(U), s)

    print ()
    """
    
    data = Build_Dataset(df_Vehicle, df_Person)
    
    print ('Features in data, with Number of Unique Values and Number of Blank Values')
    for feature in data:
        U = data[feature].unique()
        s = data[feature].isna().sum()
        print (feature, len(U), s)
    print ()
    
    """
    print ('Features in df_Vehicle with Names')
    F = []
    for feature in df_Vehicle:
        if 'NAME' in feature:
            f = feature[:-4]
            if f in data:
                F.append(f)
    F = sorted(F)
    for f in F:
        print ("        '%s'," % f)
    print ()
    """
       
    Named_Features_df_Vehicle = [
        'ACC_TYPE',
        'BDYTYP_IM',
        'BUS_USE',
        'CARGO_BT',
        'DEFORMED',
        'DR_PRES',
        'EMER_USE',
        'FIRE_EXP',
        'HAZ_CNO',
        'HAZ_INV',
        'HAZ_PLAC',
        'HAZ_REL',
#        'HITRUN_IM',
        'HIT_RUN',
        'IMPACT1',
        'IMPACT1_IM',
        'J_KNIFE',
        'MAK_MOD',
        'MAX_VSEV',
        'MDLYR_IM',
        'MOD_YEAR',
        'MXVSEV_IM',
        'M_HARM',
        'NUMINJ_IM',
        'NUMOCCS',
        'NUM_INJV',
        'PCRASH1_IM',
        'PCRASH4',
        'PCRASH5',
        'P_CRASH1',
        'P_CRASH2',
        'P_CRASH3',
        'ROLINLOC',
        'ROLLOVER',
        'SPEC_USE',
        'SPEEDREL',
        'TOWED',
        'TOW_VEH',
        'TRAV_SP',
        'VALIGN',
        'VEH_ALCH',
        'VEVENT_IM',
        'VNUM_LAN',
        'VPROFILE',
        'VSPD_LIM',
        'VSURCOND',
        'VTCONT_F',
        'VTRAFCON',
        'VTRAFWAY',
        'V_ALCH_IM',
    ]

    Feature_Names_Dict = Feature_Names(df_Vehicle, Named_Features_df_Vehicle)

    Named_Features_df_Person = [
        'HOSPITAL',
#        'MAK_MOD',
#        'SEX_IM',
#        'MAKE',
#        'BODY_TYP',
#        'PER_TYP'
#        'MOD_YEAR',
    ]

    Feature_Names_Dict.update(Feature_Names(df_Person, Named_Features_df_Person))

#    print (Feature_Names_Dict)

    Correlation_All(data, target, Feature_Names_Dict)


Main()

Import_Data()
Import_Data_Vehicle()
2016 82149
2017 97625
2018 86105
2019 96717
2020 94718
(457314, 184)

Import_Data_Person()
2016 117759
2017 138913
2018 120230
2019 135410
2020 131962
(644274, 117)


Build_Dataset()

Build_Vehicle_Dataset()

Build_Person_Dataset

Do all of the cases in data_Person appear in data_Vehicle?
True

data_Vehicle.shape
(457314, 54)
data_Person.shape
(621222, 3)
data.shape
(619027, 55)
        CASENUM  HOSPITAL  VEH_NO  ACC_TYPE  BDYTYP_IM  BODY_TYP  BUS_USE  \
0  201600014311         0       1        46          4         4        0   
1  201600014311         0       2        45          5         5        0   
2  201600014315         0       1        68         31        31        0   
3  201600014315         0       2        69         30        30        0   
4  201600014315         0       2        69         30        30        0   

   CARGO_BT  DEFORMED  DR_PRES  ...  VALIGN  VEH_ALCH  VEVENT_IM  VNUM_LAN  \
0         0         4        1  ...      

,Feature,Code,Name,Per,Corr
0,HOSPITAL,0,Not Transported for Treatment,14.4076,100.0
1,HOSPITAL,1,EMS Air,14.4076,100.0



    feature = 'VEH_NO'
    A = [
        ['0', [13,7,8,]], #  0.0275 %
        ['1', [1,]], #  54.403 %
        ['2', [9,]], #  0.0044 %
        ['3', [2,]], #  40.0519 %
        ['4', [6,10,3,5,4,11,12,14,15,]], #  5.5133 %
        ['Unknowns', []]
    ]
    data = Build_Individual_Feature_with_Dict(df_Veh, data, feature, A)



,Feature,Code,Name,Per,Corr
0,VEH_NO,13,13,0.0010,66.6667
1,VEH_NO,7,7,0.0173,17.7570
2,VEH_NO,8,8,0.0092,15.7895
3,VEH_NO,1,1,54.4030,15.0418
4,VEH_NO,9,9,0.0044,14.8148
5,VEH_NO,2,2,40.0519,14.1232
6,VEH_NO,6,6,0.0483,13.3779
7,VEH_NO,10,10,0.0015,11.1111
8,VEH_NO,3,3,4.4416,10.3582
9,VEH_NO,5,5,0.1788,9.8464



    feature = 'ACC_TYPE'
    A = [
        ['0', [61,60,51,50,53,59,52,55,58,6,54,1,10,14,16,5,2,7,8,4,0,62,3,9,89,69,41,64,66,87,90,91,]], #  21.7211 %
        ['1', [83,34,35,88,68,65,86,30,82,38,73,39,]], #  20.5914 %
        ['2', [98,25,22,11,31,77,12,40,85,24,26,32,71,81,79,29,27,43,]], #  22.3576 %
        ['3', [21,33,42,48,75,72,80,15,78,28,76,44,45,84,49,]], #  22.3708 %
        ['4', [20,67,23,74,47,70,46,93,13,92,63,36,37,]], #  12.9592 %
        ['Unknowns', [99, ]]
    ]
    data = Build_Individual_Feature_with_Dict(df_Veh, data, feature, A)



...


    feature = 'VTRAFCON'
    A = [
        ['0', [29,40,28,9,4,98,65,]], #  2.0442 %
        ['1', [0,]], #  62.2529 %
        ['2', []], #  0.0 %
        ['3', [3,]], #  24.9677 %
        ['4', [1,20,7,8,23,50,2,21,]], #  10.7352 %
        ['Unknowns', [97, 99, ]]
    ]
    data = Build_Individual_Feature_with_Dict(df_Veh, data, feature, A)



,Feature,Code,Name,Per,Corr
0,VTRAFCON,29,Unknown Regulatory Sign,0.0047,25.9259
1,VTRAFCON,40,Warning Sign,0.9904,21.4921
2,VTRAFCON,28,Other Regulatory Sign,0.1326,19.7917
3,VTRAFCON,9,Unknown Highway Traffic Signal,0.0086,18.0000
4,VTRAFCON,4,Flashing Traffic Control Signal,0.3094,17.6339
5,VTRAFCON,98,Other,0.4713,17.1795
6,VTRAFCON,65,Railway Crossing Device,0.1272,15.8752
7,VTRAFCON,0,No Controls,62.2529,15.3471
8,VTRAFCON,3,Traffic control signal(on colors) not known wh...,24.9677,12.9974
9,VTRAFCON,1,Traffic control signal (on colors) without Ped...,0.5316,12.9912



    feature = 'VTRAFWAY'
    A = [
        ['0', []], #  0.0 %
        ['1', [1,]], #  45.1904 %
        ['2', []], #  0.0 %
        ['3', [2,]], #  17.174 %
        ['4', [5,]], #  5.757 %
        ['5', [3,]], #  23.3378 %
        ['6', [6,4,0,]], #  8.541 %
        ['Unknowns', [8, 9, ]]
    ]
    data = Build_Individual_Feature_with_Dict(df_Veh, data, feature, A)



,Feature,Code,Name,Per,Corr
0,VTRAFWAY,1,"Two-Way, Not Divided",45.1904,16.8750
1,VTRAFWAY,2,"Two-Way, Divided, Unprotected Median",17.1740,14.4629
2,VTRAFWAY,5,"Two-Way, Not Divided With a Continuous Left-Tu...",5.7570,14.3864
3,VTRAFWAY,3,"Two-Way, Divided, Positive Median Barrier",23.3378,14.0477
4,VTRAFWAY,6,Entrance/Exit Ramp,2.6355,11.1775
5,VTRAFWAY,4,One-Way Trafficway,2.8350,10.1202
6,VTRAFWAY,0,Non-Trafficway or Driveway Access,3.0705,7.5867



        Feature  Code                           Name      Per   Corr
0      HOSPITAL     0  Not Transported for Treatment  14.4076  100.0
1      HOSPITAL     1                        EMS Air  14.4076  100.0
2     BDYTYP_IM    86                             86   0.0002  100.0
3      BODY_TYP    86                             86   0.0002  100.0
4       MAK_MOD  6010                           6010   0.0003  100.0
...         ...   ...                            ...      ...    ...
2509    TRAV_SP   101                            101   0.0003    0.0
2510    TRAV_SP   150                        150 MPH   0.0010    0.0
2511    TRAV_SP   139                            139   0.0003    0.0
2512    TRAV_SP   104                        104 MPH   0.0003    0.0
2513    TRAV_SP   125                        125 MPH   0.0010    0.0

[2514 rows x 5 columns]

        Feature  Code                           Name      Per      Corr
0      HOSPITAL     0  Not Transported for Treatment  14.4076  100.0000
1